In [1]:
# !pip install -r requirements.txt

In [1]:
import snowflake.snowpark.functions as F
from snowflake.snowpark.session import Session
from snowflake.snowpark.types import StructType, StructField, FloatType
from snowflake.snowpark import Session
import os
import json
import pandas as pd
import numpy as np

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib
import mlflow
import mlflow.sklearn
import mlflow.xgboost
import xgboost as xgb

In [3]:
connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "schema": "SENSOR",
    "database": "THINGSBOARD",
    "role": os.getenv("SNOWFLAKE_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
}

session = Session.builder.configs(connection_parameters).create()

In [6]:
environmental_df = session.table("co_table_predication")

In [7]:
environmental_df.show(5)

--------------------------------------------
|"TS"                 |"CO"                |
--------------------------------------------
|2024-01-01 12:00:00  |4.379166666666667   |
|2024-01-02 12:00:00  |4.270833333333335   |
|2024-01-03 12:00:00  |4.179166666666668   |
|2024-01-04 12:00:00  |4.2250000000000005  |
|2024-01-05 12:00:00  |4.175               |
--------------------------------------------



In [8]:
df =environmental_df.toPandas()

In [9]:
df.head()

,TS,CO
0,2024-01-01 12:00:00,4.379167
1,2024-01-02 12:00:00,4.270833
2,2024-01-03 12:00:00,4.179167
3,2024-01-04 12:00:00,4.225000
4,2024-01-05 12:00:00,4.175000


In [10]:
df = df.sort_values('TS').reset_index(drop=True)

In [11]:

for lag in range(1, 4):  # Lag 1, 2, 3
    df[f'CO_lag_{lag}'] = df['CO'].shift(lag)

In [12]:
df['CO_roll_mean_3'] = df['CO'].rolling(window=3).mean()
df['CO_roll_std_3'] = df['CO'].rolling(window=3).std()

In [13]:
# Extract hour and minute
df['hour'] = df['TS'].dt.hour
df['minute'] = df['TS'].dt.minute

# Cyclical encoding
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['minute_sin'] = np.sin(2 * np.pi * df['minute'] / 60)
df['minute_cos'] = np.cos(2 * np.pi * df['minute'] / 60)


In [14]:
df = df.dropna().reset_index(drop=True)

In [15]:
df.head()

,TS,CO,CO_lag_1,CO_lag_2,CO_lag_3,CO_roll_mean_3,CO_roll_std_3,hour,minute,hour_sin,hour_cos,minute_sin,minute_cos
0,2024-01-04 12:00:00,4.225000,4.179167,4.270833,4.379167,4.225000,0.045833,12,0,1.224647e-16,-1.0,0.0,1.0
1,2024-01-05 12:00:00,4.175000,4.225000,4.179167,4.270833,4.193056,0.027743,12,0,1.224647e-16,-1.0,0.0,1.0
2,2024-01-06 12:00:00,4.166667,4.175000,4.225000,4.179167,4.188889,0.031549,12,0,1.224647e-16,-1.0,0.0,1.0
3,2024-01-07 12:00:00,4.283333,4.166667,4.175000,4.225000,4.208333,0.065085,12,0,1.224647e-16,-1.0,0.0,1.0
4,2024-01-08 12:00:00,4.183333,4.283333,4.166667,4.175000,4.211111,0.063099,12,0,1.224647e-16,-1.0,0.0,1.0


In [16]:
# Define feature columns
feature_cols = [
    'CO_lag_1', 'CO_lag_2', 'CO_lag_3',
    'CO_roll_mean_3', 'CO_roll_std_3',
    'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos'
]

X = df[feature_cols]
y = df['CO']



In [17]:
import mlflow

mlflow.set_tracking_uri("https://fuzzy-goldfish-r4rr46q4jp4jf5gv6-5000.app.github.dev/")


In [18]:
mlflow.set_experiment("Time_Series_CO_Predict")


2024/11/27 13:10:52 INFO mlflow.tracking.fluent: Experiment with name 'Time_Series_CO_Predict' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/artifacts/2', creation_time=1732713052738, experiment_id='2', last_update_time=1732713052738, lifecycle_stage='active', name='Time_Series_CO_Predict', tags={}>

In [19]:
tscv = TimeSeriesSplit(n_splits=3)

# Start MLflow Run
with mlflow.start_run(run_name="Time_Series_CO_Predict") as run:
    # Define and Train the Model
    rf = RandomForestRegressor(n_estimators=200, max_depth=None, min_samples_split=2, random_state=42)
    rf.fit(X, y)

    # Predictions (Using the entire dataset for simplicity; consider using a separate test set)
    predictions = rf.predict(X)

    # Calculate Metrics
    mae = mean_absolute_error(y, predictions)
    rmse = np.sqrt(mean_squared_error(y, predictions))

    # Log Parameters
    mlflow.log_param("model", "Time_Series_CO_Predict")
    mlflow.log_param("n_estimators", 200)
    mlflow.log_param("max_depth", None)
    mlflow.log_param("min_samples_split", 2)
    mlflow.log_param("random_state", 42)

    # Log Metrics
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("RMSE", rmse)

    # Log the Model
    mlflow.sklearn.log_model(rf, "model")

    print(f"Run ID: {run.info.run_id}")
    print(f"Logged MAE: {mae:.2f}, RMSE: {rmse:.2f}")

2024/11/27 13:11:22 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

2024/11/27 13:11:25 WARNING mlflow.models.model: Model logged without a signature and input example. Ple

Run ID: b937619ec8b743cdb435fa1259fe3046
Logged MAE: 0.02, RMSE: 0.02
🏃 View run Time_Series_CO_Predict at: https://fuzzy-goldfish-r4rr46q4jp4jf5gv6-5000.app.github.dev/#/experiments/2/runs/b937619ec8b743cdb435fa1259fe3046
🧪 View experiment at: https://fuzzy-goldfish-r4rr46q4jp4jf5gv6-5000.app.github.dev/#/experiments/2


In [20]:
# Get the Run ID from the previous step
run_id = run.info.run_id

# Register the model
model_name = "Time_Series_CO_Predict"
result = mlflow.register_model(
    f"runs:/{run_id}/model",
    model_name
)

print(f"Registered Model: {result.name}, Version: {result.version}")


Successfully registered model 'Time_Series_CO_Predict'.
2024/11/27 13:11:29 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Time_Series_CO_Predict, version 1


Registered Model: Time_Series_CO_Predict, Version: 1


Created version '1' of model 'Time_Series_CO_Predict'.


In [21]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# Transition the model to 'Staging'
client.transition_model_version_stage(
    name=model_name,
    version=result.version,
    stage="Staging",
    archive_existing_versions=True
)

print(f"Model {model_name} version {result.version} transitioned to Staging.")


Model Time_Series_CO_Predict version 1 transitioned to Staging.


/tmp/ipykernel_139/1892959298.py:6: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


In [22]:
df_new

NameError: name 'df_new' is not defined

In [ ]:
import pandas as pd
import numpy as np

import mlflow
import mlflow.sklearn
mlflow.set_tracking_uri("https://legendary-dollop-7vr49x7r7v9pcxwrw-5000.app.github.dev/")

model_name = "Time_Series_CO_Model"
model_stage = "Staging"

# Load the model from the registry
model = mlflow.sklearn.load_model(
    model_uri=f"models:/{model_name}/{model_stage}"
)

print("Model loaded successfully from MLflow Model Registry.")
ts = [1672531200000, 1672531260000, 1672531320000, 1672531380000, 1672531440000]

# Sample data for CO
co = [10.5, 12.3, 9.8, 11.7, 10]

# Create the DataFrame
df_new = pd.DataFrame({'TS': ts, 'CO': co})
df_new['TS'] = pd.to_datetime(df_new['TS'], unit='ms')

# Sort by timestamp
df_new = df_new.sort_values('TS').reset_index(drop=True)
for lag in range(1, 4):
    df_new[f'CO_lag_{lag}'] = df_new['CO'].shift(lag)

# Generating rolling statistics
df_new['CO_roll_mean_3'] = df_new['CO'].rolling(window=3).mean()
df_new['CO_roll_std_3'] = df_new['CO'].rolling(window=3).std()

# Extracting time-based features
df_new['hour'] = df_new['TS'].dt.hour
df_new['minute'] = df_new['TS'].dt.minute

# Cyclical Encoding
df_new['hour_sin'] = np.sin(2 * np.pi * df_new['hour'] / 24)
df_new['hour_cos'] = np.cos(2 * np.pi * df_new['hour'] / 24)
df_new['minute_sin'] = np.sin(2 * np.pi * df_new['minute'] / 60)
df_new['minute_cos'] = np.cos(2 * np.pi * df_new['minute'] / 60)

# Drop rows with NaN values resulting from lagging and rolling
df_new= df_new.dropna().reset_index(drop=True)
feature_cols = [
            'CO_lag_1', 'CO_lag_2', 'CO_lag_3',
            'CO_roll_mean_3', 'CO_roll_std_3',
            'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos'
        ]

X_input = df_new[feature_cols]
predicted_CO = model.predict(X_input)


In [ ]:
predicted_CO

In [ ]:
latest_row

In [ ]:
X

In [ ]:
predicted_CO = model.predict(X_input)

In [ ]:
predicted_CO

In [ ]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
mlflow.set_tracking_uri("https://legendary-dollop-7vr49x7r7v9pcxwrw-5000.app.github.dev/")
model_name = "Time_Series_CO_Model"
model_version = 1  

model = mlflow.sklearn.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)
ts = [1672531200000, 1672531260000, 1672531320000, 1672531380000, 1672531440000]


co = [10.5, 12.3, 9.8, 11.7, 10]
df_new = pd.DataFrame({'TS': ts, 'CO': co})
df_new['TS'] = pd.to_datetime(df_new['TS'], unit='ms')

# Sort by timestamp
df_new = df_new.sort_values('TS').reset_index(drop=True)

# Create lag features
for lag in range(1, 4):
    df_new[f'CO_lag_{lag}'] = df_new['CO'].shift(lag)

# Generate rolling statistics
df_new['CO_roll_mean_3'] = df_new['CO'].rolling(window=3).mean()
df_new['CO_roll_std_3'] = df_new['CO'].rolling(window=3).std()

# Extract time-based features
df_new['hour'] = df_new['TS'].dt.hour
df_new['minute'] = df_new['TS'].dt.minute

# Cyclical Encoding
df_new['hour_sin'] = np.sin(2 * np.pi * df_new['hour'] / 24)
df_new['hour_cos'] = np.cos(2 * np.pi * df_new['hour'] / 24)
df_new['minute_sin'] = np.sin(2 * np.pi * df_new['minute'] / 60)
df_new['minute_cos'] = np.cos(2 * np.pi * df_new['minute'] / 60)

# Drop rows with NaN values resulting from lagging and rolling
df_new = df_new.dropna().reset_index(drop=True)

# Define feature columns
feature_cols = [
    'CO_lag_1', 'CO_lag_2', 'CO_lag_3',
    'CO_roll_mean_3', 'CO_roll_std_3',
    'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos'
]

# Prepare input features
X_input = df_new[feature_cols]

# Make predictions
predicted_CO = model.predict(X_input)

print("Predictions:", predicted_CO)


In [ ]:
##  ds is timestamp
##
import pandas as pd
import numpy as np
from prophet import Prophet

import pandas as pd
import numpy as np

import mlflow
import mlflow.sklearn

# Set the MLflow tracking URI
mlflow.set_tracking_uri("https://legendary-dollop-7vr49x7r7v9pcxwrw-5000.app.github.dev/")

model_name = "Time_Series_CO_Model"
model_version = 1  # Specify the exact version you want to load

# Load the model by specifying its version
model = mlflow.sklearn.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)

print("Model loaded successfully from MLflow Model Registry.")



df = pd.DataFrame()
df['ds'] = pd.to_datetime(input_x, unit='ms')
df['hour'] = df['ds'].dt.hour
df['minute'] = df['ds'].dt.minute
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['minute_sin'] = np.sin(2 * np.pi * df['minute'] / 60)
df['minute_cos'] = np.cos(2 * np.pi * df['minute'] / 60)


df['CO'] = np.array(input_y)
for lag in range(1, 4):
    df_new[f'CO_lag_{lag}'] = df_new['CO'].shift(lag)

# Generate rolling statistics
df['CO_roll_mean_3'] = df['CO'].rolling(window=3).mean()
df['CO_roll_std_3'] = df['CO'].rolling(window=3).std()

df = df.dropna().reset_index(drop=True)

feature_cols = [
    'CO_lag_1', 'CO_lag_2', 'CO_lag_3',
    'CO_roll_mean_3', 'CO_roll_std_3',
    'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos'
]

X_input = df[feature_cols]
model.predict(X_input)
forecast = model.predict(future)
output_y = forecast['yhat'].tolist()
print(f"result: {output_y}")
return output_y